In [ ]:
import numpy as np
import matplotlib.pyplot as plt


$ \large \text{Consider the function: } \\ h(x)= 512(x_2-x_1^2)^2+(4-x_1)^2 $

$\large \text{Answer: minimizer of function h(x) is [4,16] and minimum function value is 0 } $

$\large \text{[R] Can you use the exact line search idea to find a closed form expression for solving minη q(x + ηp) where} 
\\ \text{p not equal to 0 is a descent direction at x? If yes, find the closed form expression for optimal value of η. If you cannot
find closed form solution, explain the reasons.}$

In [ ]:
def evalf(x):
  assert len(x)==2 and type(x) is np.ndarray
  return 512*(x[1]-x[0]**2)**2 + (4-x[0])**2

In [ ]:
def evalg(x):
  assert len(x)==2  and type(x) is np.ndarray
  return np.array([1024*(x[1]-x[0]**2)*(-2*x[0])-2*(4-x[0]),1024*(x[1]-x[0]**2)])

In [ ]:
def compute_steplength_exact(x):
  y_1,y_2,a_1,a_2,a_3,a_4=[0 for i in range(6)]
  y_1,y_2,a_1,a_2,a_3,a_4= [np.longdouble(y_1),np.longdouble(y_2),np.longdouble(a_1),np.longdouble(a_2),np.longdouble(a_3),np.longdouble(a_4)]
  y_1=1024*(x[0]*2)-1024*(x[1])
  y_2=8-2*x[0]-2048*(x[0]**3)+2048*x[0]*x[1]
  a_1=(2048*(y_2*4))/(10*8)
  a_2=(1024*(y_2*2)*(y_1-2*x[0]*y_2) + 2048*(y_2*2)*(y_1-2*x[0]*y_2))/(10*8)
  a_2=-a_2
  a_3=(1024*(y_1-2*x[0]*y_2)*(y_1-2*x[0]*y_2)-2048*(y_2*2)*(x[1]-(x[0]**2)) + 2*(y_2*2))/(10*8)
  a_4=(1024*(y_1-2*x[0]*y_2)*(x[1]-(x[0]**2)) -2*y_2*(4-x[0]))/(10**8)
  root=np.roots([a_1,a_2,a_3,a_4])
  for i in root:
    if 3*a_1*(i**2) + 2*a_2*i +a_3 > 0:
      return i

$\large \text{we obtained step length value of exact line search expression value of alpha so we are able to do it with exact line search method.} $

In [40]:
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  
  
  alpha = alpha_start
  p=-gradf
  while evalf(x+alpha*p)> evalf(x)+gamma*alpha*(np.dot(evalg(x).transpose(),p)):
    alpha=alpha*rho
  return alpha

In [ ]:
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2

$ \large \text{[R] With starting point x0 = (100, 100) and τ = 10−10, we will now study the behavior of the backtracking line} \\ \text{
search algorithm for different choices of α0.Take} γ = ρ = 0.5. \text{Try α0 ∈ {1, 0.9, 0.75, 0.6, 0.5, 0.4, 0.25, 0.1, 0.01}.} \\ 
\text{For each α0, record the final minimizer, final objective function value and number of iterations taken by the} \\ \text{
gradient descent algorithm with backtracking line search to terminate. Prepare a plot where the number of iterations is plotted against α0 values. Comment on the observations. Comment about the minimizers and} \\ $


$ \text{
objective function values obtained for different choices of the α0 values. If you have implemented exact line } $
search, check and comment if for any α0 value, gradient descent with backtracking line search takes lesser
number of iterations when compared to the gradient descent procedure with exact line search.$ 

$\large \text{I will bound the number of iterations by 10000} $

In [ ]:
def find_minimizer_cls(start_x, tol):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  x = start_x
  g_x = evalg(x)
  


  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    step_length = compute_steplength_exact(x) #call the new function you wrote to compute the steplength
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
    if k>10000:
      break
    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k 

In [ ]:
def find_minimizer_bls(start_x, tol, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  x = start_x
  g_x = evalg(x)
  alpha_start = args[0]
  rho = args[1]
  gamma = args[2]
  print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
    if k>10000:
      break
    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k 

In [ ]:
start=np.array([100,100])
tol=1e-10

print('Solution using closed-form expression')
x_opt_cls,number_of_iter_cls=find_minimizer_cls(start,tol)
print('Minimizer of function:',x_opt_cls)
print('Minuimum function value:',evalf(x_opt_cls))
print('number of iterations:',number_of_iter_cls)
print('---------------------------------------------------------------------')
#check what happens when you call find_minimzer using backtracking line search
print('Solution using backtracking line search')
alpha_list=np.array([1,0.9,0.75,0.6,0.5,0.4,0.25,0.1,0.01])
for alpha in alpha_list:
  start=np.array([100,100])
  tol=1e-10
  x_opt_bls,number_of_iter_bls = find_minimizer_bls(start, tol, alpha, 0.5,0.5)
  print('Minimizer of function:',x_opt_bls)
  print('Minuimum function value:',evalf(x_opt_bls))
  print('number of iterations:',number_of_iter_bls)
  print('\n---------------------------------------------------------\n')

Solution using closed-form expression


<ipython-input-12-8c4bed1c1b82>:7: RuntimeWarning: overflow encountered in long_scalars
  a_2=(1024*(y_2*2)*(y_1-2*x[0]*y_2) + 2048*(y_2*2)*(y_1-2*x[0]*y_2))/(10*8)
<ipython-input-12-8c4bed1c1b82>:9: RuntimeWarning: overflow encountered in long_scalars
  a_3=(1024*(y_1-2*x[0]*y_2)*(y_1-2*x[0]*y_2)-2048*(y_2*2)*(x[1]-(x[0]**2)) + 2*(y_2*2))/(10*8)


Minimizer of function: [ 2.21496135e+09 -1.10747052e+07]
Minuimum function value: 1.2323514115394172e+40
number of iterations: 10001
---------------------------------------------------------------------
Solution using backtracking line search
Params for Backtracking LS: alpha start: 1.0 rho: 0.5  gamma: 0.5
Minimizer of function: [ 10.0476072  100.95498684]
Minuimum function value: 36.57372294589531
number of iterations: 10001

---------------------------------------------------------

Params for Backtracking LS: alpha start: 0.9 rho: 0.5  gamma: 0.5
Minimizer of function: [ 10.04766087 100.95613638]
Minuimum function value: 36.57441661039108
number of iterations: 10001

---------------------------------------------------------

Params for Backtracking LS: alpha start: 0.75 rho: 0.5  gamma: 0.5
Minimizer of function: [ 10.04637453 100.93015577]
Minuimum function value: 36.55878053071092
number of iterations: 10001

---------------------------------------------------------

Params for B

6. [R] With starting point x0 = (100, 100) and τ = 10−10, we will now study the behavior of the backtracking line
search algorithm for different choices of ρ. Take α = 1, γ = 0.5. Try ρ ∈ {0.9, 0.75, 0.6, 0.5, 0.4, 0.25, 0.1, 0.01}.
For each ρ, record the final minimizer, final objective function value and number of iterations taken by the
gradient descent algorithm with backtracking line search to terminate. Prepare a plot where the number of
iterations is plotted against ρ values. Comment on the observations. Comment about the minimizers and
objective function values obtained for different choices of the ρ values. If you have implemented exact line
search, check and comment if for any ρ value, gradient descent with backtracking line search takes lesser
number of iterations when compared to the gradient descent procedure with exact line search.

In [44]:
print('Solution using closed-form expression')
x_opt_cls,number_of_iter_cls=find_minimizer_cls(start,tol)
print('Minimizer of function:',x_opt_cls)
print('Minuimum function value:',evalf(x_opt_cls))
print('number of iterations:',number_of_iter_cls)
print('---------------------------------------------------------------------')
#check what happens when you call find_minimzer using backtracking line search
print('Solution using backtracking line search')
alpha_list=np.array([0.9,0.75,0.6,0.5,0.4,0.25,0.1,0.01])
for alpha in alpha_list:
  start=np.array([100,100])
  tol=1e-10
  x_opt_bls,number_of_iter_bls = find_minimizer_bls(start, tol, 1, alpha,0.5)
  print('Minimizer of function:',x_opt_bls)
  print('Minuimum function value:',evalf(x_opt_bls))
  print('number of iterations:',number_of_iter_bls)
  print('\n---------------------------------------------------------\n')

Solution using closed-form expression


<ipython-input-12-8c4bed1c1b82>:7: RuntimeWarning: overflow encountered in long_scalars
  a_2=(1024*(y_2*2)*(y_1-2*x[0]*y_2) + 2048*(y_2*2)*(y_1-2*x[0]*y_2))/(10*8)
<ipython-input-12-8c4bed1c1b82>:9: RuntimeWarning: overflow encountered in long_scalars
  a_3=(1024*(y_1-2*x[0]*y_2)*(y_1-2*x[0]*y_2)-2048*(y_2*2)*(x[1]-(x[0]**2)) + 2*(y_2*2))/(10*8)


Minimizer of function: [ 2.21496135e+09 -1.10747052e+07]
Minuimum function value: 1.2323514115394172e+40
number of iterations: 10001
---------------------------------------------------------------------
Solution using backtracking line search
Params for Backtracking LS: alpha start: 1 rho: 0.9  gamma: 0.5


<ipython-input-10-78b251d24ac9>:3: RuntimeWarning: overflow encountered in long_scalars
  return 512*(x[1]-x[0]**2)**2 + (4-x[0])**2


KeyboardInterrupt: ignored